## Fake News Classifier Using LSTM

Dataset: https://www.kaggle.com/c/fake-news/data#

In [8]:
import pandas as pd

In [9]:
df=pd.read_csv('/content/train.csv',error_bad_lines=False,engine='python')

<ipython-input-9-28ea674a6d43>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df=pd.read_csv('/content/train.csv',error_bad_lines=False,engine='python')
Skipping line 2826: unexpected end of data


In [10]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [11]:
df.shape

(2824, 5)

In [12]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [13]:
df.isnull().sum()

id          0
title      76
author    284
text        7
label       0
dtype: int64

In [14]:
###Drop Nan Values
df=df.dropna()


In [15]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [16]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [17]:
## Get the Dependent features
y=df['label']

In [18]:
X.shape

(2464, 4)

In [19]:
y.shape

(2464,)

In [20]:
import tensorflow as tf

In [21]:
tf.__version__

'2.12.0'

In [22]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [23]:
### Vocabulary size
voc_size=5000

### Onehot Representation

In [24]:
messages=X.copy()

In [32]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [26]:
messages

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
2819,2819,ISIS Chainsaw Massacre: Nine Christian Youths ...,admin,Ads ISIS Chainsaw Massacre: Nine Christian You...
2820,2820,Would a Trump Presidency Mean Economic Disaste...,Neil Irwin,If Donald Trump were elected president and put...
2821,2821,Trump to CNN: ’You Are Fake News’ - Breitbart,Ian Hanchett,"During a press conference on Wednesday, Dona..."
2822,2822,Monsanto Weed Killer Roundup Faces New Doubts ...,Danny Hakim,"The reputation of Roundup, whose active ingred..."


In [35]:
messages.reset_index(inplace=True)

In [36]:
messages

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...,...
2459,2819,2819,ISIS Chainsaw Massacre: Nine Christian Youths ...,admin,Ads ISIS Chainsaw Massacre: Nine Christian You...
2460,2820,2820,Would a Trump Presidency Mean Economic Disaste...,Neil Irwin,If Donald Trump were elected president and put...
2461,2821,2821,Trump to CNN: ’You Are Fake News’ - Breitbart,Ian Hanchett,"During a press conference on Wednesday, Dona..."
2462,2822,2822,Monsanto Weed Killer Roundup Faces New Doubts ...,Danny Hakim,"The reputation of Roundup, whose active ingred..."


In [29]:
import nltk
import re
from nltk.corpus import stopwords

In [30]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [37]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer ##stemming purpose
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [38]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [ ]:
corpus[1]

In [39]:
onehot_repr=[one_hot(words,voc_size)for words in corpus]
onehot_repr

[[1104, 833, 2532, 4005, 4760, 4615, 2025, 680, 1746, 3103],
 [2997, 3573, 92, 3009, 3666, 2144, 3535],
 [1759, 2396, 3207, 3192],
 [4882, 1121, 4545, 541, 3188, 2986],
 [348, 3666, 1839, 1512, 1925, 1698, 3666, 2087, 4369, 1063],
 [1721,
  1848,
  683,
  4817,
  2662,
  1652,
  1155,
  2987,
  4249,
  4243,
  3875,
  1225,
  2010,
  2238,
  3535],
 [3851, 2433, 1385, 4770, 4107, 609, 2708, 2001, 3538, 716, 3992],
 [1672, 3172, 543, 4180, 2212, 2836, 1652, 3122, 3538, 716, 3992],
 [11, 2954, 2762, 4279, 430, 4712, 3291, 3532, 1652, 840],
 [267, 70, 4275, 1553, 2277, 4892, 1691, 2120],
 [710, 380, 476, 2953, 1457, 1383, 401, 1813, 3993, 2339, 3105],
 [541, 3261, 4760, 4712, 1652, 2212],
 [3039, 4726, 1280, 3147, 910, 16, 3136, 1764, 3201],
 [4775, 3360, 3585, 23, 3979, 4946, 4146, 3538, 716, 3992],
 [3957, 558, 2707, 1064, 900, 3538, 716, 3992],
 [609, 1943, 3587, 1202, 3315, 3866, 2981, 4321, 1392, 2871],
 [1469, 399, 3573],
 [2998, 1717, 4121, 971, 1652, 399, 4847, 3535],
 [3562, 641,

In [ ]:
corpus[1]

In [40]:
onehot_repr[1]

[2997, 3573, 92, 3009, 3666, 2144, 3535]

### Embedding Representation

In [41]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='post',maxlen=sent_length)
print(embedded_docs)

[[1104  833 2532 ...    0    0    0]
 [2997 3573   92 ...    0    0    0]
 [1759 2396 3207 ...    0    0    0]
 ...
 [1652 1987 2821 ...    0    0    0]
 [4014 3366 2836 ...    0    0    0]
 [1170 1063  832 ...    0    0    0]]


In [42]:
embedded_docs[1]

array([2997, 3573,   92, 3009, 3666, 2144, 3535,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [43]:
embedded_docs[0]

array([1104,  833, 2532, 4005, 4760, 4615, 2025,  680, 1746, 3103,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [57]:
## Creating model
embedding_vector_features=40 ##features representation
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 20, 40)            200000    
                                                                 
 lstm_2 (LSTM)               (None, 100)               56400     
                                                                 
 dense_2 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [45]:
len(embedded_docs),y.shape

(2464, (2464,))

In [58]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [47]:
X_final.shape,y_final.shape

((2464, 20), (2464,))

In [59]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### Model Training

In [60]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
26/26 [==============================] - 7s 182ms/step - loss: 0.6834 - accuracy: 0.5691 - val_loss: 0.6566 - val_accuracy: 0.7457
Epoch 2/10
26/26 [==============================] - 4s 168ms/step - loss: 0.3843 - accuracy: 0.8648 - val_loss: 0.3542 - val_accuracy: 0.8391
Epoch 3/10
26/26 [==============================] - 2s 69ms/step - loss: 0.1414 - accuracy: 0.9576 - val_loss: 0.3350 - val_accuracy: 0.8698
Epoch 4/10
26/26 [==============================] - 1s 49ms/step - loss: 0.0453 - accuracy: 0.9915 - val_loss: 0.6614 - val_accuracy: 0.8292
Epoch 5/10
26/26 [==============================] - 1s 45ms/step - loss: 0.0097 - accuracy: 0.9976 - val_loss: 0.6427 - val_accuracy: 0.8317
Epoch 6/10
26/26 [==============================] - 1s 45ms/step - loss: 0.0037 - accuracy: 0.9994 - val_loss: 0.6810 - val_accuracy: 0.8219
Epoch 7/10
26/26 [==============================] - 1s 22ms/step - loss: 0.0057 - accuracy: 0.9970 - val_loss: 0.9351 - val_accuracy: 0.8034
Epoch 8/10


### Adding Dropout

In [50]:
# from tensorflow.keras.layers import Dropout
# ## Creating model
# embedding_vector_features=40
# model=Sequential()
# model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
# model.add(Dropout(0.3))
# model.add(LSTM(100))
# model.add(Dropout(0.3))
# model.add(Dense(1,activation='sigmoid'))
# model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

### Performance Metrics And Accuracy

In [61]:
y_pred=model.predict(X_test)

26/26 [==============================] - 0s 3ms/step


In [62]:
y_pred=np.where(y_pred > 0.5, 1,0) ##AUC ROC Curve

In [53]:
from sklearn.metrics import confusion_matrix

In [63]:
confusion_matrix(y_test,y_pred)

array([[393,  71],
       [ 81, 269]])

In [64]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.8132678132678133

In [65]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.83      0.85      0.84       464
           1       0.79      0.77      0.78       350

    accuracy                           0.81       814
   macro avg       0.81      0.81      0.81       814
weighted avg       0.81      0.81      0.81       814

